# Customizing Apice-py Pipeline
This Jupyter Notebook tutorial will guide you through the process of creating and customizing a preprocessing pipeline using the apice-py framework.

## 1. Prerequisites

Before we begin, make sure you have the following prerequisites in place:

- Python 3.12 or latest installed
- Jupyter Notebook installed
- Environment with apice-py requirements installed
- apice-py module (folder)


## 2. Importing Libraries

In [ ]:
import sys
import os

import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt  # Import Matplotlib for data visualization

import mne  # Import MNE-Python for EEG data handling
import mne.baseline  # Import MNE-Python baseline module for baseline correction

# Import APICE (assuming it's in a custom location)
sys.path.append(r"apice")

from apice.pipeline import *  # Import APICE pipeline components
from apice.artifacts_rejection import *  # Import APICE artifacts rejection tools
from apice.io import *  # Import APICE input/output functions
from apice.filter import *  # Import APICE filtering methods


## 3. Directories and Input Parameters

In [ ]:
# Base directory where your main directory is located
BASE_DIR = os.getcwd()

# Directory for input data
INPUT_DIR = r".../input"

# Directory for output data
OUTPUT_DIR = r".../output"

# Selection method ( 1 - Run it for all the files found and overwrite previous output files (default)
# 2 - Run it only for the new files
# 3 - Run specific files. Space key + enter key to stop the input prompt.)
SELECTION_METHOD = 1

# Montage (information regarding the sensor locations, - built_in mne montage or - electrode layout file)
MONTAGE = r"electrode_layout/GSN-HydroCel-128.sfp"

# Event keys for segmentation (array of event types relative to the epochs)
EVENT_KEYS_FOR_SEGMENTATION = ['Icue', 'Ieye', 'Iout'] 

# Event time window for segmentation (start and end time of the epochs in seconds)
EVENT_TIME_WINDOW = [-1.600, 2.200] 

# Baseline time window for segmentation (time interval to consider as baseline when applying baseline correction of epochs, in seconds)
BASELINE_TIME_WINDOW = [-1.600, 0]

# Flag to indicate whether to process data by event type
BY_EVENT_TYPE = True

# Flag to save preprocessed raw data
SAVE_PREPROCESSED_RAW = True

# Flag to save segmented data
SAVE_SEGMENTED_DATA = True  

# Flag to save evoked response data
SAVE_EVOKED_RESPONSE = True

# Flag to save log files
SAVE_LOG = False 

# Number of core used in the computation (-1 to use all the available, faster computation)
N_JOBS = -1


## 4. Running APICE on an IDE
Incorporating the complete APICE pipeline into your code within an Integrated Development Environment (IDE) is straightforward. Here's an example:

In [ ]:
import apice.pipeline

# Run the APICE pipeline with specified parameters
apice.pipeline.run(
    input_dir=INPUT_DIR,                                        # Directory containing your input data
    output_dir=OUTPUT_DIR,                                      # Directory where the output data will be saved
    data_selection_method=SELECTION_METHOD,                     # Data selection method (e.g., 1)
    event_keys_for_segmentation=EVENT_KEYS_FOR_SEGMENTATION,    # Event keys for segmentation
    event_time_window=EVENT_TIME_WINDOW,                        # Event time window, e.g., [-0.2, 2.2]
    baseline_time_window=BASELINE_TIME_WINDOW,                  # Baseline time window, e.g., [-0.2, 0]
    montage=MONTAGE,                                            # EEG electrode montage, e.g., "GSN-HydroCel-128"
    by_event_type=BY_EVENT_TYPE,                                # Flag to process data by event type
    save_preprocessed_raw=SAVE_PREPROCESSED_RAW,                # Flag to save preprocessed raw data
    save_segmented_data=SAVE_SEGMENTED_DATA,                    # Flag to save segmented data
    save_evoked_response=SAVE_EVOKED_RESPONSE,                  # Flag to save evoked response data
    save_log=SAVE_LOG,                                          # Flag to save log files
    n_jobs=N_JOBS                                               # Number of parallel jobs
)

# Alternatively, you can provide the parameters directly
# apice.pipeline.run(
#     input_dir=INPUT_DIR,
#     output_dir=OUTPUT_DIR,
#     data_selection_method=1,
#     event_keys_for_segmentation=["Iout", "Icue"],
#     event_time_window=[-0.2, 2.2],
#     baseline_time_window=[-0.2, 0],
#     montage="GSN-HydroCel-128",
#     by_event_type=True,
#     save_preprocessed_raw=True,
#     save_segmented_data=True,
#     save_evoked_response=True,
#     save_log=False
#     n_jobs=-1
# )


## 5. Customizing the APICE pipeline

One of the key advantages of using the APICE framework is its flexibility and customizability. You can choose the order and components of the preprocessing pipeline to tailor it to your specific data and analysis needs.

### 5.1 Import Data

In [ ]:
# Define the filename of the EEG data you want to import
file_name = 'data_example.set'

# Create the full path to the input EEG data file by joining the INPUT_DIR and file_name
# INPUT_DIR = r"..\input"
full_path = os.path.join(INPUT_DIR, file_name)

# Import the EEG data using the APICE library
from apice.io import Raw
# EEG = Raw.import_raw(full_path, montage="GSN-HydroCel-129")
EEG = Raw.import_raw(full_path, montage=MONTAGE)

In [ ]:
# Import the necessary MNE-Python module for EEG data visualization
import mne

# Create a figure to visualize the raw EEG data
fig = mne.viz.plot_raw(
    raw=EEG,                        # The EEG data you want to visualize
    start=0,                        # The start time (in seconds) for visualization
    duration=100,                   # The duration (in seconds) of the data to visualize
    n_channels=len(EEG.ch_names),   # Number of EEG channels (automatically determined)
    scalings=50e-6,                 # Scalings for the EEG data (adjust as needed)
    clipping=None                   # Clipping options (None means no clipping)
)

In [ ]:
# visualize the electrode layout
EEG.plot_sensors(show_names=True)

### 5.2 Filter

In [ ]:
# Define the minimum and maximum frequencies for the filter
fmin = 0.2  # Minimum frequency (Hz)
fmax = 40   # Maximum frequency (Hz)

# Import the Filter module from the APICE library
from apice.filter import Filter

# Apply the filter to the EEG data
# This filters the EEG data to retain frequencies between fmin and fmax
Filter(EEG, high_pass_freq=fmin, low_pass_freq=fmax)

# Rescale EEG data using MNE-Python's baseline correction
EEG._data = mne.baseline.rescale(
    EEG._data,  # EEG data to rescale
    EEG.times,  # Time information
    baseline=(None, None),  # Baseline period (None means the entire recording)
    mode='mean',  # Rescaling mode (mean)
    copy=False  # Modify the EEG data in place
)

In [ ]:
# Create a figure to visualize the raw EEG data
fig = mne.viz.plot_raw(
    raw=EEG,                        # The EEG data you want to visualize
    start=0,                        # The start time (in seconds) for visualization
    duration=100,                   # The duration (in seconds) of the data to visualize
    n_channels=len(EEG.ch_names),   # Number of EEG channels (automatically determined)
    scalings=50e-6,                 # Scalings for the EEG data (adjust as needed)
    clipping=None                   # Clipping options (None means no clipping)
)

In [ ]:
# Set Matplotlib to use the 'qt' backend for interactive plots
# %matplotlib qt

# Compute the PSD of EEG data within the specified frequency range (fmin to fmax)
psd = EEG.compute_psd(fmin=fmin, fmax=fmax)

# Plot the PSD using the 'qt' backend for interactive plotting
fig = psd.plot()

# Switch back to the 'inline' backend for non-interactive plots
# %matplotlib inline

### 5.3 Artifact Detection

#### 5.3.1 Detect artifacts using the default artifact detection algorithm:

In [ ]:
# Create a copy of the EEG data for processing (so we don't have to import it again)
EEG_copy = EEG.copy()

# Detect and mark artifacts in the EEG data using the APICE pipeline
apice.pipeline.detect_artifacts(EEG_copy)

# Create a separate copy of EEG data with detected artifacts for later use
EEG_with_artifacts = EEG_copy.copy()

In [ ]:
# Import the necessary module for visualizing artifact structure
from apice.artifacts_structure import DefineBTBC

# Create a figure to visualize the artifact structure
fig = DefineBTBC.plot_artifact_structure(
    EEG_copy,               # EEG data with marked artifacts
    color_scheme='jet'  # Color scheme for visualization
)
fig.show()

#### 5.3.1 Detecting Bad Electrodes, Motion Artifacts, and Jump Artifacts

In [ ]:
# Create a copy of the EEG data for processing
EEG_copy = EEG.copy()

# Detect nonfunctional electrodes using the BadElectrodes function
# This function identifies electrodes that may not be functioning correctly.
apice.artifacts_rejection.BadElectrodes(EEG_copy)

# Define Bad Times, Bad Channels (BTBC) and plot the rejection matrix
# The DefineBTBC function helps identify segments of data with potential issues.
from apice.artifacts_structure import DefineBTBC
DefineBTBC(EEG_copy)

# It also generates a rejection matrix for visualization.
# Use 'plot_rejection_matrix' to True to display the rejection matrix.
fig = DefineBTBC.plot_artifact_structure(EEG_copy, artifact='BC', time_step=100) # artifact = 'all', 'BCT', 'BT', 'BC'

In [ ]:
# Create a copy of the EEG data for processing
EEG_copy = EEG.copy()

# Detect motion artifacts using the Motion function
# This function is used to identify motion artifacts in the EEG data.
apice.artifacts_rejection.Motion(EEG_copy)

# Define Bad Times, Bad Channels (BTBC) and plot the rejection matrix
# The DefineBTBC function helps identify segments of data with potential issues.
from apice.artifacts_structure import DefineBTBC
DefineBTBC(EEG_copy)

# It also generates a rejection matrix for visualization.
# Use 'plot_rejection_matrix' to True to display the rejection matrix.
fig = DefineBTBC.plot_artifact_structure(EEG_copy, artifact='all') # artifact = 'all', 'BCT', 'BT', 'BC'

#### 5.3.3 Detecting specific artifacts

You can also use apice to detect artifacts, for example, those that are higher that a certain amplitude threshold.

In [ ]:
# Create a copy of the EEG data for processing
EEG_copy = EEG.copy()

# Set the artifacts matrix of the EEG data using the Artifacts class
from apice.artifacts_structure import Artifacts
EEG_copy.artifacts = Artifacts(EEG_copy)

# Detect high amplitude signals as artifacts
from apice.artifacts_detection import Amplitude
Amplitude(EEG_copy, thresh=500 * 1e-6)

# Visualize the artifact structure using DefineBTBC
from apice.artifacts_structure import DefineBTBC
fig = DefineBTBC.plot_artifact_structure(EEG_copy)

In [ ]:
# Create a copy of the EEG data for processing
EEG_copy = EEG.copy()

# Set the artifacts matrix of the EEG data using the Artifacts class
from apice.artifacts_structure import Artifacts
EEG_copy.artifacts = Artifacts(EEG_copy)

# Detect artifacts based on channel amplitude correlation
from apice.artifacts_detection import ChannelCorr
ChannelCorr(EEG_copy, time_window=4, time_window_step=2, thresh=0.4)

# Visualize the artifact structure using DefineBTBC
from apice.artifacts_structure import DefineBTBC
fig = DefineBTBC.plot_artifact_structure(EEG_copy)

In [ ]:
# Create a copy of the EEG data for processing
EEG_copy = EEG.copy()

# Set the artifacts matrix of the EEG data using the Artifacts class
from apice.artifacts_structure import Artifacts
EEG_copy.artifacts = Artifacts(EEG_copy)

# Detect high amplitude signals using a relative threshold
from apice.artifacts_detection import Amplitude
Amplitude(EEG_copy, use_relative_thresh=True)

# Visualize the artifact structure using DefineBTBC
from apice.artifacts_structure import DefineBTBC
fig = DefineBTBC(EEG_copy, plot_rejection_matrix=True)

Please refer to artifacts_detection.py for the list of algorithms you can use.

### 5.4 Artifact Correction

You can use the artifact correction algorithm, the same idea as in Artifact Detection codes.

#### 5.4.1 Correcting artifacts using the default APICE pipeline

In [ ]:
# Create a copy of the EEG data with artifacts for correction
EEG_copy = EEG_with_artifacts.copy()

# Correct artifacts using the default APICE pipeline
apice.pipeline.correct_artifacts(EEG_copy)

# Visualize the artifact structure after correction
from apice.artifacts_structure import DefineBTBC
fig = DefineBTBC(EEG_copy, plot_rejection_matrix=True)

#### 5.4.2 Correcting Artifacts separately

In [ ]:
# Create a copy of the EEG data with artifacts for correction
EEG_copy = EEG_with_artifacts.copy()

# Import artifact correction modules
from apice.artifacts_correction import TargetPCA, SegmentSphericalSplineInterpolation, ChannelsSphericalSplineInterpolation

# Apply Target PCA per Electrode for artifact correction
TargetPCA(EEG_copy, config=True)  # config=True to use default parameters

# Rescale EEG data using MNE-Python's baseline correction
EEG._data = mne.baseline.rescale(
    EEG_copy._data,  # EEG data to rescale
    EEG.times,  # Time information
    baseline=(None, None),  # Baseline period (None means the entire recording)
    mode='mean',  # Rescaling mode (mean)
    copy=False  # Modify the EEG data in place
)

# Apply filtering to the EEG data
apice.filter.Filter(EEG_copy, high_pass_freq=0.2, low_pass_freq=[])

# Visualize the artifact structure after correction
from apice.artifacts_structure import DefineBTBC
apice.artifacts_structure.DefineBTBC(EEG_copy, config=True)  # config=True to use default parameters

In [ ]:
# Spline Interpolation
SegmentSphericalSplineInterpolation(EEG_copy, min_good_time=2)
EEG._data = mne.baseline.rescale(EEG_copy._data, EEG_copy.times, baseline=(None, None), mode='mean', copy=False)
apice.filter.Filter(EEG_copy, high_pass_freq=0.2, low_pass_freq=[])
apice.artifacts_structure.DefineBTBC(EEG_copy, config=True)

In [ ]:
# Apply Channels Spline Interpolation for whole channels
from apice.artifacts_correction import ChannelsSphericalSplineInterpolation
ChannelsSphericalSplineInterpolation(EEG_copy, config=True)

# Detect motion artifacts
from apice.artifacts_rejection import Motion
Motion(EEG_copy, type=2, keep_rejected_previous=False, config=True)

# Visualize the artifact structure after correction and rejection
from apice.artifacts_structure import DefineBTBC
apice.artifacts_structure.DefineBTBC(
    EEG_copy,
    plot_rejection_matrix=True,
    config=True
)

Results are different from 5.4.1 because we did not use the same parameters (`config=False`)

### 5.5 Segmentation

In [ ]:
# Segment EEG data into epochs and define evoked responses
Epochs, Evokeds = apice.pipeline.segment_data(
    EEG_copy,                                   # EEG data to segment
    event_keys=EVENT_KEYS_FOR_SEGMENTATION,     # Event keys for segmentation
    tmin=-0.2,                                  # Start time of epochs (relative to events)
    tmax=2.2,                                   # End time of epochs (relative to events)
    by_event_type=True,                         # Segment by event type
    n_jobs=-1
)

# If you don't need the evoked response
# Epochs, _ = apice.pipeline.segment_data(
#     EEG_copy,  # EEG data to segment
#     event_keys=EVENT_KEYS_FOR_SEGMENTATION,  # Event keys for segmentation
#     tmin=-0.2,  # Start time of epochs (relative to events)
#     tmax=2.2,  # End time of epochs (relative to events)
#     by_event_type=True  # Segment by event type
# )


In [ ]:
# Visualize the segmented EEG epochs
fig = Epochs.plot(events=Epochs.events, event_id = Epochs.event_id, scalings=50e-6)

In [ ]:
# Plot the event-related potential (ERP) for the first event key
fig = Evokeds[0].plot()